Kd Fitting
===

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from champ import intensity_array, seqtools
from champ.kd import KdFitIA
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from IPython.display import HTML

In [ ]:
date = ''
out_bname = 'LDA'
min_conc = 4000  # Lowest concentration of protein (pM)
max_conc = 1024000  # Highest concentration of protein (pM)

custom_results_dir = os.path.join('results', date, 'custom')
custom_fig_dir = os.path.join('figs', date, 'custom')
inten_array_fpath = os.path.join(custom_results_dir, '{}_intensity_scores.txt'.format(out_bname))
print 'Date:', date

## Setup

In [ ]:
def mm_name(seq):
    mms = []
    for i, (c1, c2) in enumerate(zip(kdf.target, seq)):
        if c1 != c2:
            pos = i + 1 - 3
            if pos <= 0:
                pos -= 1
            mms.append('{}{}{}'.format(c1, pos, c2))
    return ','.join(mms)

In [ ]:
IA = intensity_array.IntensityArray()
IA.parse_intensities_file(inten_array_fpath)
new_concentration_list = [conc for conc in IA.course_trait_list if min_conc <= conc <= max_conc]
IA = IA.subIA(course_traits=new_concentration_list)

In [ ]:
print(IA.course_trait_list)

In [ ]:
single_ham_seqs = seqtools.get_mismatch_seqs(IA.target, 1)

In [ ]:
sinham_IA = IA.subIA(seqs=single_ham_seqs)

## Just Single Hamming Dist

In [ ]:
kdf = KdFitIA(sinham_IA)
kdf.find_Imin_and_background_noise()
kdf.find_Imax()

In [ ]:
print(kdf.nM_concentrations)

In [ ]:
example_seqs = [kdf.target, kdf.neg_control_target] + random.sample(single_ham_seqs, 2)
labels = ['Target', 'Neg Control'] + map(mm_name, example_seqs[2:])
kdf.example_plots(example_seqs, labels)

In [ ]:
kdf.fit_all_Kds()

In [ ]:
kdf.write_results(custom_results_dir, out_bname + '_sinham')

## Error Analysis

In [ ]:
kdf.all_error_analysis_and_figs(out_dir=custom_fig_dir, out_bname=out_bname)

## All Seqs

In [ ]:
kdf = KdFitIA(IA)
kdf.find_Imin_and_background_noise()
kdf.find_Imax()

In [ ]:
kdf.fit_all_Kds()

In [ ]:
kdf.write_results(custom_results_dir, out_bname)

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')